## Tasks

![tasks](https://cdn-images-1.medium.com/max/1600/1*q7L9L7Nb6-c3VbHvs3auXQ.png)

## Object detection

### Какие метрики использовать?

<details><summary>Спойлер</summary>
<p>

#### 1. IoU - intersection over union или Jaccard coefficient

Мера того, насколько хорошо ground truth пересекается с предсказаниями модели

<img src="https://www.pyimagesearch.com/wp-content/uploads/2016/09/iou_equation.png" alt="IoU" width=300px/>

$$ \Large IoU = \dfrac{TP}{TP + FP + FN} $$


#### 2. mean average precision at different IoU

Cчитается путем усреднения AP для разных порогов IoU, обычно `range(0.5, 1.0, 0.05)`. Такой подход поощряет модели, которые дают лучшую локализацию объекта

В отличие от задачи сегментации, где мы смотрели на пересечения масок, в задаче детекции мы смотрим на пересечение bounding boxes


```
The metric sweeps over a range of IoU thresholds, at each point calculating an average precision value. The threshold values range from 0.5 to 0.95 with a step size of 0.05: (0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95).

In other words, at a threshold of 0.5, a predicted object is considered a "hit" if its intersection over union with a ground truth object is greater than 0.5.

At each threshold value t, a precision value is calculated based on the number of true positives (TP), false negatives (FN), and false positives (FP) resulting from comparing the predicted object to all ground truth objects.

A true positive is counted when a single predicted object matches a ground truth object with an IoU above the threshold.
A false positive indicates a predicted object had no associated ground truth object.
A false negative indicates a ground truth object had no associated predicted object.

The average precision of a single image is then calculated as the mean of the above precision values at each IoU threshold.
```

Больше про IoU и mAP можно почитать здесь:

- https://medium.com/@timothycarlen/understanding-the-map-evaluation-metric-for-object-detection-a07fe6962cf3
- https://medium.com/@jonathan_hui/map-mean-average-precision-for-object-detection-45c121a31173
- https://github.com/rafaelpadilla/Object-Detection-Metrics
- https://www.jeremyjordan.me/evaluating-image-segmentation-models/

</p>
</details>

### А какой loss?

<details><summary>Спойлер</summary>
<p>

C loss дела обстоят сложнее, чем в задаче сегментации

Нужно учесть и класс найденного объекта, и его bounding box, поэтому loss состоит из нескольких частей

Подробнее дальше

</p>
</details>

### Faster RCNN

`Faster R-CNN` является логическим продолжением архитектур `R-CNN` и `Fast R-CNN`

Краткое напоминание о предках:

#### R-CNN

[paper](https://arxiv.org/abs/1311.2524)

- selective search для поиска регионов (Regions of Interest, ROI)
- reshape, на вход CNN для извлечения признаков
- SVM по этим признакам классифицирует регионы
- bounding box regressor по тем же признакам предсказывает bounding boxes (неожиданно то как) регионов

**Selective Search**

Graph-based segmentation ([paper](http://cs.brown.edu/people/pfelzens/papers/seg-ijcv.pdf))

<img src="https://www.learnopencv.com/wp-content/uploads/2017/09/breakfast-300x200.jpg" alt="selective_search1"/>
<img src="https://www.learnopencv.com/wp-content/uploads/2017/09/breakfast_oversegment-300x200.jpg" alt="selective_search2"/>

1. Добавляем все bounding boxes, соответствующие имеющимся сегментам, в RoI
2. Группируем смежные сегменты на основе метрки близости (обычно это color, texture, size and shape similarities)
3. Повторяем с 1 до тех пор, пока боксом не станет все изображение
4. Берем M самых поздних боксов (можно сделать сортировку по времени слегка рандомной, умножив индекс бокса на случайное число от 0 до 1)

<img src="https://www.learnopencv.com/wp-content/uploads/2017/09/hierarchical-segmentation-1.jpg" alt="selective_search1"/>

**Проблемы**

- Медленно и ресурсозатратно: ищутся регионы (обычно 2000 для изображения), для каждого извлекаем N признаков с помощью CNN, потом еще SVM и bbox regressor. Секунды или даже минуты на одно предсказания
- не end-to-end, по сути имеем 3 модели
- сложно и долго учить

<img src="https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2018/10/Screenshot-from-2018-10-08-15-06-33.png" alt="rcnn" width=600px/>


#### Fast R-CNN

[paper](https://arxiv.org/abs/1504.08083)

- изображение на вход CNN, которая возвращает RoI
- `RoI pooling layer` для приведения всех регионов к одному размеру
- fully connected layers с двумя выходами: linear для регрессии боксов и linear + softmax для классификации

**Проблемы**

- все еще медленно, хоть и не так, как при использовании`R-CNN`. Решили проблему с прогоном каждого региона через CNN, но selective search остался. А это секунды на каждое изображение
- все еще сложно и долго учить

<img src="https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2018/10/Screenshot-from-2018-10-08-15-47-18.png" alt="fast-rcnn" width=600px/>


А теперь про `Faster R-CNN`

#### Faster R-CNN

- изображение на вход CNN, которая возвращает feature maps
- `Region proposal network (RPN)` получает эти feature maps и возвращает RoI с objectness score
- `RoI pooling layer` для приведения всех регионов к одному размеру
- fully connected layers с двумя выходами: linear для регрессии боксов и linear + softmax для классификации


**RPN**

`RPN` проходит скользящим окном по feature map, генерируя для каждого окна $k$ anchor boxes разных размеров и формы. Для каждого из них `RPN` предсказывает вероятность, что anchor - это объект без учета класса ($2k$ скоров для softmax и $k$ для sigmoid) и bounding box для уточнения размера anchor'a ($4k$ координат: центр, ширина, высота)


**RoI pooling**

[Хорошее объяснение](https://deepsense.ai/region-of-interest-pooling-explained/)

- Каждый RoI делится на N равных секций, N - размерность выхода `RoI pooling` слоя
- Ищется максимум в каждой секции и добавляется в выходной вектор

<img src="https://cdn-sv1.deepsense.ai/wp-content/uploads/2017/02/roi_pooling-1.gif" alt="RoI_pooling_1" width=500px/>

**Как учить?**

1. отдельно `RPN`, отдельно финальные классификатор и регрессор
2. все вместе (быстрее с тем же качеством)

**Используемый loss**

$$ \Large L = \dfrac{1}{N_{cls}} (L_{classification} + \lambda \dfrac{1}{N_{reg}} p_i^* L_{regression}) $$

- $ L_{classification} $  - бинарный logloss
- $ L_{regression} = smooth_{L1}(t_i - t_i^*) $, где $t_i$ - вектор координат предсказанного бокса, а $t_i^*$ - ground truth бокса

<img src="https://cdn-images-1.medium.com/max/800/1*AglMn7WQAj_p0vJHqt0hxg.png" alt="smooth-L1" width=300px/>


- $\lambda = 10$ в исходной реализации, но можно подбирать на валидации для конкретной задачи. Авторы статьи используют значение 10, чтобы сделать веса лоссов примерно одинаковыми, т.к. $N_{cls} = 256$, а $N_{reg} ~ 2400$. Также в статье приведены результаты для разных значений $\lambda$
- $p_i^* = 1$, если anhor - это объект. Поэтому получается, что regression loss не учитывается для не объектов
 
 
**Проблемы**

- все еще довольно медленно. Хоть и быстрее предков, но не real-time
- все еще сложно и долго учить
- вроде бы и end-to-end, но "костыльный"

[paper](https://arxiv.org/abs/1506.01497)

[хорошая PyTorch реализация](https://github.com/jwyang/faster-rcnn.pytorch)

![faster-rcnn](https://lilianweng.github.io/lil-log/assets/images/faster-RCNN.png)


### SSD

[paper](https://arxiv.org/abs/1512.02325)

![ssd](https://lilianweng.github.io/lil-log/assets/images/SSD-architecture.png)

`SSD` использует похожий принцип, что и `region proposal network (RPN)` в
`Faster R-CNN`: для предсказаний используется фиксированный набор боксов (default bounding boxes), что похоже на использование anchor boxes в RPN.

Однако в `SSD` сразу вычисляется score для каждого класса в каждом боксе, вместо испрользования `RoI pooling layer` и обучения классификатора поверх него. Такой подход проще и быстрее.

Для поиска bounding box, так же, как и в `Faster R-CNN`, решается задача регрессии для центра $(cx, cy)$ default bounding box $d$, его ширины $w$ и высоты $h$


**Используемый loss**

$$ \Large L = \dfrac{1}{N} (L_{confidence} + \alpha L_{localization}) $$

- $L_{confidence}$ - confidence loss: насколько модель точно предсказывает класс объекта. Это softmax loss для нескольких классов
- $L_{localization}$ - localization loss: насколько близок bounding box найденного объекта к ground truth. Это **Smooth L1 loss** между предсказанным bounding box и ground truth
- $N$ - количество default boxes, если N = 0, то $ L = 0 $
- $\alpha$ - гиперпараметр и должен подбираться на валидации, однако можно выбирать равным 1 по принципу *good enough*

Теперь разберемся с составными частями функции потерь

**$L_{confidence}$**

$$ \Large L_{confidence} = - \sum_{i \in Pos}^N x_{ij}^p \log(\bar{c_i}^p) - \sum_{i \in Neg} \log(\bar{c_i}^p) $$
$$ \bar{c_i}^p = \dfrac{\exp({c_i}^p)}{\sum_{\bar{p}}\exp({c_i}^\bar{p})} $$

Здесь negative класс - это background boxes, $ x_{ij}^p = 1 $, если $i$-й box сматчился с $j$-м ground truth box категории $p$

**$L_{localization}$**

$$ \Large L_{localization} = \sum_{i \in Pos}^N \quad \sum_{m \in \{ cx, cy, h, w \} } x_{ij}^k \quad smooth_{L1} (l_i^m - \bar{g_j^m}) $$

$$ \bar{g_j^cx} = (g_j^cx - d_i^cx) / d_i^w  \qquad  \bar{g_j^cy} = (g_j^cy - d_i^cy) / d_i^h $$
$$ \bar{g_j^w} = \log \bigg( \dfrac{g_j^w}{d_i^w} \bigg)  \qquad  \bar{g_j^h} = \log \bigg( \dfrac{g_j^h}{d_i^h} \bigg) $$

$L_{localization}$ считается только для тех боксов, которые сматчились с ground truth (positive box).


**Проблемы:**

- `SSD` плохо предсказывает маленькие объекты, т.к. работает с изображениями 300x300 и 512x512 (слегка решает проблему, но не до конца). Однако, существует ряд улучшений, позволяющих лучше работать с маленькими изображениями.
- быстрее `Faster R-CNN`, но хуже по качеству
- на текущий момент существуют архитектуры, которые, скорее всего, работают быстрее и дают лучшее качество модели (`RetinaNet`, `YOLO`)

### Другие архитектуры

#### FPN

[paper](https://arxiv.org/abs/1612.03144)

Скорее не отдельная архитектура, а backbone для других архитектур

<img src="https://cdn-images-1.medium.com/max/800/1*D_EAjMnlR9v4LqHhEYZJLg.png" alt="retina_net" width=600px/>

**The bottom-up pathway**

- Высокоуровневые признаки несут больше семантики. FPN - способ реконструировать high resolution признаки из семантически богатых слоев
- Lateral Connection позволяют модели лучше связать исходные feature maps и реконструированные признаки + это аналог skip connections, улучшающих обучению
- Один из способов решить проблему с детекцией маленьких объектов

#### RetinaNet

[paper](https://arxiv.org/abs/1708.02002)

`RetinaNet` использует `FPN` над `ResNet`

<img src="https://lilianweng.github.io/lil-log/assets/images/retina-net.png" alt="retina_net" width=800px/>

**Focal loss**

Одной из проблем задачи object detection является дисбаланс между фоном (background) и объектами (foreground). `Focal loss` призван решить это проблему путем добавления веса сложным примерам, например, примерам с кусками объектов или сложной текстурой

<img src="https://lilianweng.github.io/lil-log/assets/images/focal-loss.png" alt="focal_loss" width=500px/>


#### YOLO

[project page](https://pjreddie.com/darknet/yolo/)

<img src="https://cdn-images-1.medium.com/max/1200/1*d4Eg17IVJ0L41e7CTWLLSg.png" alt="yolo_v3" width=800px/>

<img src="https://cdn-images-1.medium.com/max/800/1*YpNE9OQeshABhBgjyEXlLA.png" alt="yolo_v3_comparison" width=600px/>


Работает быстро и вроде как быстро, но, говорят, сложно прикрутить к своей задаче

### Используемые трюки

#### Non-Maximum Suppression

Что делать, если модель нашла много bounding boxes для одного объекта?

Выход: `Non-Maximum Suppression`

- сортируем все боксы по confidence score
- убираем боксы, в которых не уверены
- далее жадный алгоритм пока у нас остаются боксы
    - выбираем самый уверенный - это предсказание
    - выкидываем из оставшихся все боксы, у которых $IoU > 0.5$ с выбранным, т.е. сильно пересекающиеся
    - выбираем следующее предсказание
    
![NMS](https://lilianweng.github.io/lil-log/assets/images/non-max-suppression.png)

#### Hard Negative Mining

Выше мы уже говорили про сложные примеры, теперь рассмотрим другой подход решения этой проблемы. 

Разделим боксы без объектов на *easy negative* (содержат только фон) и *hard negative* (содержат часть объектов или сложную текстуру). Моделям часто сложно распознать *hard negative* от объектов, поэтому мы можем добавлять такие false positive примеры во время тренировки для улучшения классификации

Выбирать *hard negative* примеры можно из тех примеров train, на которых модель ошибается или не уверена

### Сравнение разных архитектур

[Источник](https://medium.com/@jonathan_hui/object-detection-speed-and-accuracy-comparison-faster-r-cnn-r-fcn-ssd-and-yolo-5425656ae359)

![perforance](https://cdn-images-1.medium.com/max/1200/1*7dJTcEv7vYAQyHbQ8QAZUA.png)

![fps](https://cdn-images-1.medium.com/max/1200/1*EQKFp_c6jMYcDZbIwbEOzA.png)

Что почитать про object detection
- http://zoey4ai.com/2018/05/12/deep-learning-object-detection/
- https://www.analyticsvidhya.com/blog/2018/10/a-step-by-step-introduction-to-the-basic-object-detection-algorithms-part-1/
- https://lilianweng.github.io/lil-log/2017/12/31/object-recognition-for-dummies-part-3.html
- https://lilianweng.github.io/lil-log/2018/12/27/object-detection-part-4.html
- https://medium.com/@jonathan_hui/object-detection-series-24d03a12f904
- https://medium.com/@smallfishbigsea/faster-r-cnn-explained-864d4fb7e3f8